In [ ]:
import scipy.special as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize,least_squares,Bounds,basinhopping
from scipy.special import gamma
import matplotlib.cm as cm
from Boruvka_Mod import Graph
import utm
import scipy.interpolate as spline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import math

In [ ]:
#PUT CELL ABOVE TARGET FUNCTION
data = pd.read_csv("nsea86.m77t", sep='\t')
mindepth = min(data['CORR_DEPTH'])
print('Shallowest depth:', mindepth, 'm')

In [ ]:
#User defined values

onshore_connection_point = None # add cost of a cable from here to substation??
substation_coords = None
maximumturbinespertree = 5 # number of turbines allowed along a single cable from the substation.
budget = 60000000 # Budget in £ to build the wind farm, accounting for construction costs.

In [ ]:
###CALCULATES THE INITIAL NUMBER OF TURBINES THE OPTIMISATION SHOULD START WITH.

maxnodespertree = maximumturbinespertree # the +1 for the substation node is accommodated for in the code

import math

FixedCost=600000###

a = 300 # minimum distances turbines can be from one another.
b = 3000 # distance that turbines no longer affect eachother? Where would I find this?
         # If model changes this value needs to change


minareaperturbine = (1/2)*(math.sqrt(3))*(a**2)
    
# all these values need to be linked to target function definitions of costs

LandCostperTurbine=minareaperturbine*(17245/10000) # £1.7245/m^2 of land?
CostperTurbine=3000000###
MaintenanceCostperTurbine=2*1563720###
MinDepthCostPerTurbine = 3000000*(0.0002*(float(mindepth)**2) - 0.0002*(float(mindepth)) + 0.9459)-3000000
#y=3000000(0.0002x^2-0.0002x+0.9459)-3000000
print('MinDepthCostPerTurbine',MinDepthCostPerTurbine)

nturb = (budget-FixedCost)/(CostperTurbine+LandCostperTurbine+MaintenanceCostperTurbine+MinDepthCostPerTurbine)   #need to include depth cost


nturb = math.floor(nturb) # rounds down as cannot have fraction of a turbine
minarea = minareaperturbine*nturb

print('min area per turbine', minareaperturbine)
print('min area =', minarea, 'm^2')
print('max number of turbines =', nturb)

maxarea = nturb*(1/2)*(math.sqrt(3))*(b**2) # careful this isn't bigger than the entire size of the scaled nsea86 sea floor.
                                            # max area of 30million currently reaches that limit therefore need to reduce the
                                            # /36 scale? or reduce the maxarea?
print('max area =', maxarea, 'm^2')

# Center the coordinate system around the substation (0,0). Get this working first. To improve this you could specify
# a coastline line equation that the turbines cannot go past meaning extra turbines are forced offshore.

#There's currently a problem of knowing where to start placing turbines

In [ ]:
# place substation at (0,0).
# place 1st turbine at vector (a,0) from substation.
# place 2nd turbine at vector (0.5a,0.5aroot3) from prev
# place 3rd turbine at vector (-a,0) from prev
# place 4th turbine at vector (-a,0) from prev

import math


positionlist = np.empty((nturb+1,2))
turbineposition = np.array([[0,0]])
positionlist[0,:] = turbineposition



oddcounter = 0
evencounter = 0
i = 0
k = 0

while k<nturb+1: 
    i += 1
    if (i%2)!= 0: ## i is odd
        oddcounter+=1
        if (oddcounter%2)!= 0: #oddcounter is odd
            for j in range(oddcounter):
                k += 1
                if k == nturb+1:
                    break
                #print('h')
                shift = np.array([[a, 0]])
                turbineposition = np.add(turbineposition, shift)
                #positionlist = np.concatenate((positionlist,turbineposition),axis=0)
                positionlist[k,:] = turbineposition
                
        else:
            for j in range(oddcounter):
                k += 1
                if k == nturb+1:
                    break
                #print('h')
                shift = np.array([[-a,0]])
                turbineposition = np.add(turbineposition, shift) 
                positionlist[k,:] = turbineposition
                
                                            
    else:
        evencounter+=1
        if (evencounter%2)!= 0: #evencounter is odd
            for j in range(evencounter):
                k += 1
                if k == nturb+1:
                    break
                #print('d')
                shift = np.array([[0.5*a,0.5*a*math.sqrt(3)]])
                turbineposition = np.add(turbineposition, shift) 
                positionlist[k,:] = turbineposition
                
        else:
            for j in range(evencounter):
                k += 1
                if k == nturb+1:
                    break
                #print('d')
                shift = np.array([[-0.5*a,-0.5*a*math.sqrt(3)]])
                turbineposition = np.add(turbineposition, shift) 
                positionlist[k,:] = turbineposition
                
import sys
np.set_printoptions(threshold=sys.maxsize)
print(positionlist)        

print(len(positionlist))


#plt.xlim([-1500, 1500])
#plt.ylim([-1500, 1500])
fig, ax = plt.subplots()
ax.scatter(positionlist[:,0],positionlist[:,1],c='red')
ax.scatter(0,0)
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')


#could make substation a different colour in scatter plot  

In [ ]:
# Combine this cell and the cell below into a function that clusters the turbines.

def clustering_algorithm(positionlist,nturb,maxnodespertree):

    k = math.ceil(nturb/maxnodespertree) #number of clusters


    from minmax_kmeans import minsize_kmeans

    [turbines, centres] = minsize_kmeans(positionlist[1:,:] ,k ,min_size = 1, max_size = maxnodespertree)

    label = np.concatenate((np.array([-1]),turbines))

    u_labels = np.unique(label)

    

   

    a = 0

    indiceslist = np.empty((k,maxnodespertree), dtype=np.int8)




    for i in range(k):
        indices = [i for i, x in enumerate(label) if x == a]

        if len(indices) < maxnodespertree:
            noofemptys = maxnodespertree - len(indices)
            for l in range(noofemptys):
                indices.append(int(-100))
    
        indiceslist[i,:] = (indices)

        a += 1
    
    
    
        
    
    arr = np.zeros((k,1),dtype=np.int8)   
    indiceslist = np.concatenate((arr,indiceslist),axis=1)
    print(indiceslist)
    
    return indiceslist,u_labels,label
indiceslist,u_labels,label = clustering_algorithm(positionlist,nturb,maxnodespertree)

fig,ax = plt.subplots()
for i in u_labels:
    ax.scatter(positionlist[label == i , 0] , positionlist[label == i, 1] , label = i)
ax.legend()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')
plt.show()

In [ ]:
# creates input needed for MST, similar recreation of cell 4. (Could make cell 4 a function then call it here).
def geom_analysis(positionlist,indiceslist):
    
    #Arrays for pairwise distances and angles
    #Angle 0: x directıon. Angle pi/2: y directıon

    distance = [None] * math.ceil(nturb/maxnodespertree)
    angle = [None] * math.ceil(nturb/maxnodespertree)
    distance_global = np.zeros((nturb,nturb))
    angle_global = np.zeros((nturb,nturb))
    coords = positionlist[1:,:]
#Calculate global distances 
    for i in range(0,nturb):
        distance_global[i,:]=np.sqrt(np.square(coords[i,0]-coords[:,0])+np.square(coords[i,1]-coords[:,1]))
        #print(np.shape(distance_global))
        #print(np.shape(coords))
        angle_global[i,:]=np.arctan2(coords[:,1]-coords[i,1],coords[:,0]-coords[i,0])
        distance_global[i,i]=1e10
        #Rotate angles so that north=0 and convert to degrees
    angle_global=-np.rad2deg(angle_global)+270
    for x in range(0,math.ceil(nturb/maxnodespertree)):
    
        #nturbintree = np.count_nonzero(indiceslist[x,:]) #rows count from 0 to ...
        nturbintree = np.count_nonzero(indiceslist[x,:] > -1)


    
    
        distance[x]=np.zeros((nturbintree,nturbintree))
        angle[x]=np.zeros((nturbintree,nturbintree))
    
    
        for i in range(0,nturbintree):

            for j in range(0,nturbintree):
                
                distance[x][i,j] = np.sqrt(np.square(positionlist[int(indiceslist[x,i]),0]-positionlist[int(indiceslist[x,j]),0]) + np.square(positionlist[int(indiceslist[x,i]),1]-positionlist[int(indiceslist[x,j]),1]))
                angle[x][i,j]=np.arctan2(positionlist[int(indiceslist[x,j]),1]-positionlist[int(indiceslist[x,i]),1],positionlist[int(indiceslist[x,j]),0]-positionlist[int(indiceslist[x,i]),0])
          
                distance[x][i,i]=1e10
        
        #Rotate angles so that north=0 and convert to degrees
        angle[x]=-np.rad2deg(angle[x])+270      #FIXXXXXXXXXXXXXXXXXXXXX!!!!!!!!!!!!!!!!
      
    return distance,angle,distance_global,angle_global ### angle = geom_analysis(positionlist)[1], but angle still needs converting to deg as above

In [ ]:
# determines minimum amount of cable needed to connect all turbines.
# creates lower bound, ignoring redundancy and load balancing, etc.

#ADD 0,0 AS A NODE FOR EACH GRAPH!

def Minimum_Spanning_Tree(distance,indiceslist):

    MSTweight = [None] * math.ceil(nturb/maxnodespertree)
   
    for x in range(0,math.ceil(nturb/maxnodespertree)):
        
        #nturbintree = np.count_nonzero(indiceslist[x,:]) #rows count from 0 to ...
        nturbintree = np.count_nonzero(indiceslist[x,:] > -1)
    
        g=Graph(nturbintree)
        
        for i in range(0,nturbintree):
            for j in range(i+1,nturbintree):
                g.addEdge(i,j ,int(distance[x][i,j]*1000))
        
                
        MSTweight[x]=g.boruvkaMST()

    return MSTweight

In [ ]:
# the Boruvka_mod code requires graphs with vertices 0....n, which is provided by the index value on indices list.
# When this is plotted, the values within the correct indiceslist row are then used to reference the index within
# positionlist to be plotted.
#Summeddistancetosub, min_index = sub_connection(positionlist)

MSTweight=Minimum_Spanning_Tree(geom_analysis(positionlist, indiceslist)[0],indiceslist)

fig, ax = plt.subplots()
      
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')


ax.scatter(positionlist[:,0],positionlist[:,1],c='red')
links = [None] * math.ceil(nturb/maxnodespertree)
MSTWeightSum = 0


for a in range(0,math.ceil(nturb/maxnodespertree)):
    MSTWeightSum += MSTweight[a][0]
    #p = positionlist[indiceslist[a, min_index[w][0][0]],0],0
    #q = positionlist[indiceslist[a, min_index[w][0][0]],1],0
    #ax.plot(p,q, '--k')
    
    print('each tree weight', 'tree',a+1 ,MSTweight[a][0])
    links[a] = MSTweight[a][1]
    for b in links[a]:
         
        u = positionlist[int(indiceslist[a,b[0]])][0], positionlist[int(indiceslist[a,b[1]])][0]
        
        v = positionlist[int(indiceslist[a,b[0]])][1], positionlist[int(indiceslist[a,b[1]])][1]
        

        ax.plot(u,v, c='black')
        ax.scatter(0,0,c='blue')

#print('Combined substation to tree link cable lengths:', Summeddistancetosub, 'm')        
print('MSF total weight', MSTWeightSum, 'm')

In [ ]:
print(data)

In [ ]:
#PUT CELL ABOVE TARGET FUNCTION

#coordinates are approximate, explain further

data['easting'] = data.apply(lambda row: utm.from_latlon(row['LAT'], row['LON'])[0], axis=1)
data['northing'] = data.apply(lambda row: utm.from_latlon(row['LAT'], row['LON'])[1], axis=1)

zero_east = min(data['easting']) + (max(data['easting']) - min(data['easting']))/2
zero_north = min(data['northing']) + (max(data['northing']) - min(data['northing']))/2    

#print('max north =', max(data['northing']))
#print('min north =', min(data['northing']))
#print('zero_north =', zero_north)

data['centered_easting'] = data['easting'] - zero_east
data['centered_northing'] = data['northing'] - zero_north



print(data)


In [ ]:
fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')

ax.scatter(data['centered_easting'],data['centered_northing'])
#plt.plot(data['centered_easting'],data['centered_northing'])

plt.show()

In [ ]:
# sample data from x = -50000 to 250000 and y = -90000 to 110000 to get a ratio of x:y 6:4 (same as 4x6km grid needed)

fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')

data = data[(data['centered_easting'] >=-108000) & (data['centered_easting'] <= 108000) & (data['centered_northing']>=-72000) & (data['centered_northing']<= 72000) & (data['CORR_DEPTH']<=1000) & (data['CORR_DEPTH']>=-1000)]

#print(useful_data)
ax.scatter(data['centered_easting'],data['centered_northing'])
#plt.plot(data['centered_easting'],data['centered_northing'])

plt.show()



In [ ]:
#squeeze data into 6x4km grid

fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')

data['centered_easting'] = data['centered_easting']/15
data['centered_northing'] = data['centered_northing']/15
ax.scatter(data['centered_easting'],data['centered_northing'])
#plt.plot(data['compressed_easting'],data['compressed_northing'])

plt.show()



In [ ]:
fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')


tcf = ax.tricontourf(data['centered_easting'], data['centered_northing'], data['CORR_DEPTH'])
cbar = plt.colorbar(tcf)
cbar.set_label('Depth (m)')
ax.set_title('Unsmoothed sea floor topography using discrete depth values');


In [ ]:
smooth = spline.SmoothBivariateSpline(data['centered_easting'], data['centered_northing'], data['CORR_DEPTH'], s = 25000)
fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')

tcf = ax.tricontourf(data['centered_easting'], data['centered_northing'], smooth(data['centered_easting'], data['centered_northing'], grid=False))
cbar = plt.colorbar(tcf)
cbar.set_label('Depth (m)')
ax.set_title('Smoothed continuous sea floor topography');

#cbar.set_label('Depth(m)')

In [ ]:
def depthvalues(positionlist):
#    smooth = spline.SmoothBivariateSpline(data['centered_easting'], data['centered_northing'], data['CORR_DEPTH'], s = 25000)
    DepthPerTurbine = [None] * (len(positionlist)-1)
    DepthCostPerTurbine = [None] * (len(positionlist)-1)
    DepthCostAllTurbines = 0

    for i in range(0,len(positionlist)-1):
        DepthPerTurbine[i] = spline.SmoothBivariateSpline.ev(smooth, positionlist[i+1,0], positionlist[i+1,1])
        DepthCostPerTurbine[i] = MinDepthCostPerTurbine = 3000000*(0.0002*(float(DepthPerTurbine[i])**2) - 0.0002*(float(DepthPerTurbine[i])) + 0.9459)-3000000
        
        
        DepthCostAllTurbines += DepthCostPerTurbine[i]
    mindepth=float(min(DepthPerTurbine))
    maxdepth=float(max(DepthPerTurbine))
    #print('DepthPerTurbine', DepthPerTurbine)   
    #print('DepthCostPerTurbine', DepthCostPerTurbine)
    print('DepthCostAllTurbines', DepthCostAllTurbines)
    return DepthCostAllTurbines, mindepth, maxdepth
depthvalues(positionlist)

In [ ]:
mindepth = (depthvalues(positionlist))[1]
maxdepth = (depthvalues(positionlist))[2]



fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')
ax.set_xlim([min(positionlist[:,0])-100, max(positionlist[:,0])+100])
ax.set_ylim([min(positionlist[:,1])-100, max(positionlist[:,1])+100])


links = [None] * k
MSTWeightSum = 0

for a in range(0,math.ceil(nturb/maxnodespertree)):
    MSTWeightSum += MSTweight[a][0]
    #p = positionlist[indiceslist[a, min_index[w][0][0]],0],0
    #q = positionlist[indiceslist[a, min_index[w][0][0]],1],0
    #plt.plot(p,q, '--k')
    print('each tree weight', 'tree',a+1 ,MSTweight[a][0])
    links[a] = MSTweight[a][1]
    for b in links[a]:
         
        u = positionlist[int(indiceslist[a,b[0]])][0], positionlist[int(indiceslist[a,b[1]])][0]
        
        v = positionlist[int(indiceslist[a,b[0]])][1], positionlist[int(indiceslist[a,b[1]])][1]
        
        plt.plot(u,v,c='black')

print('MSF total weight', MSTWeightSum)

levels = np.linspace(mindepth-3,maxdepth+3,24) # Change to min(smooth) and max(smooth) somehow

tcf = ax.tricontourf(data['centered_easting'], data['centered_northing'], smooth(data['centered_easting'], data['centered_northing'], grid=False), levels=levels)
#plt.imshow(tcf, vmin=16, vmax=40)
cbar = plt.colorbar(tcf)
cbar.set_label('Depth (m)')
ax.set_title('Plot showing sea floor topography and turbines with cables');

ax.scatter(positionlist[:,0],positionlist[:,1],c='red')
ax.scatter(0,0,c='blue')
#cbar.mappable.set_clim(60,70)


In [ ]:
### Sets model parameters
v0=8.0
nwind = 33
wind=np.arange(254,287)
#b=np.array((1,5,1000))
#b=np.array(( 1.40002561e-01,   8.51478121e+00,   2.62606729e+03))

# 3 model parameters below:

# 0th: how much power the turbine removes at the centre of the peak of the power distribution.
        # UNITS: [N/A]
# 1st: how wide the angle of effect is. UNITS: degrees
# 2nd: up to how far back the effect takes place (approx. 2.6km) UNITS: [m]
model=np.array((1.39998719e-01, 8.51483871e+00, 2.62613638e+03))

ws=2.0 #weibull scale factor
wei_gamma=gamma(1.+1./ws)

Pr = 2*10**6 #Rated Power for Horns Rev 1 Turbines. Max power output [Watts]
Vc = 4.0 #Cut-in Velocity. Starts producing energy at wind speed of 4m/s [m/s]
Vr = 15.0 #Rated Velocity. Starts producing max energy at 15m/s
Vf = 25.0 #Cut-off Velocity. Turbines cut out at wind speeds of 25m/s to prevent damage to the turbines.
k = 2.0 #Weibull shape parameter
(Pr,Vc,Vr,k)

#interpolation parameters
dvel=1.5 #[m/s]
dang=5. #[degrees]

In [ ]:
### (gives wind direction and velocity at HornsRev1).

v=np.loadtxt('hornsrev_data_all.txt') # assigns data to variable v.

vxref=v[:,3] # (INDEXING STARTS FROM 0). Third column of data is velocity of wind in x direction.
vyref=v[:,4] # (INDEXING STARTS FROM 0). Fourth column of data is velocity of wind in y direction.
angles=v[:,0] # Zeroth column of data is wind angle.
vmean=np.sqrt(np.square(vxref)+np.square(vyref)) # Uses pythagoras to find the wind magnitude + direction for each location.

# print(vmean)
vmean=pd.DataFrame(vmean) # organises vmean into labelled data structure
# print(vmean)

vref=pd.DataFrame()

vmean['angle']=angles # add another column to vmean (angle)
vref=vref.append(vmean) # add empty pandas data frame

vref=vref.groupby('angle')
vref.groups
vref.describe()
vref.get_group(260).iat[50,0] # data manipulation to group by angle etc.
#vref.iat[5,0]
#vref.groupby('angle')

In [ ]:
# ### Imports, manipulates and displays coordinates of all turbines at Horns Rev 1.

# # in coords matrix the first index references the turbine, the second index references said coordinate's value (0=x, 1=y).

# coords_in = np.loadtxt('coords.txt')


# # CLARIFY WHAT THESE COORDS ARE WITH PRINT: LOWER LEFT COORDS BELOW: and print a gap between outputs

# print(min(coords_in[:,0]), min(coords_in[:,1])) # finds and displays most west and most south coordinates of turbines
# # (note: not necessarily 1 turbine that represemts x and y values, just finds reference frame for wind farm). 

# # sets xmin and ymin to these lower left most coordinate values.
# xmin = min(coords_in[:,0])
# ymin = min(coords_in[:,1])

# # sets centre point halfway between the min and max coords:
# xzero = (max(coords_in[:,0])-xmin)/2
# yzero = (max(coords_in[:,1])-ymin)/2
# # print(xzero, yzero)

# # transforms coordinates so they're centred around the origin of the coordinates system:
# coords_x=coords_in[:,0]-xzero-xmin
# coords_y=coords_in[:,1]-yzero-ymin
# coords=[coords_x,coords_y]

# #print(isinstance(coords))




# print (coords) # displays array of manipulated coordinates.


# coords=np.array(coords).T # transposes coordinates
# nturb=np.shape(coords)[0] # counts total number of turbines by putting array into a tuple and counting entries.




# plt.scatter(coords[:,0], coords[:,1]) # plots turbines as a scatter plot (x,y).



#Arrays for pairwise distances and angles
#Angle 0: x direction. Angle pi/2: y directıon

# calculates distance and angle between each pair of turbines:

distance=np.zeros((nturb,nturb)) # 2x2 matrix of distances between turbines i and j where distance i-i and distance j-j = 0
# as they're distances to themselves: i-i i-j
                                 #    j-i j-j

angle=np.zeros((nturb,nturb)) # same as above but for angles between turbines.



for i in range(0,nturb):
   
    # 80x80 matrices as there's 80 turbines at Horns Rev 1.
    
    # squares y distance and x distance then sqrt to find overall distance between 2 turbines.
    distance[i,:]=np.sqrt(np.square(positionlist[i+1,0]-positionlist[1:,0])+np.square(positionlist[i+1,1]-positionlist[1:,1]))
    # same as above using arctan2 whilst giving correct quadrant (between 2 turbines).
    angle[i,:]=np.arctan2(positionlist[1:,1]-positionlist[i+1,1],positionlist[1:,0]-positionlist[i+1,0])
# Rotate angles so that north=0 and convert to degrees (and clockwise instead of anticlockwise)


angle=-np.rad2deg(angle)+270
#angle[0,1]

# Rotating angles to wind direction
windangle=5 # degrees
rotangles=np.mod(angle-windangle+180,360)-180

# creates list of turbines with largest to shortest distances between them.
(np.sort(distance))[0,2]

In [ ]:
# imports wind direction (meteorological data where north = 0 degrees)
import pandas as pd
directiondata = pd.read_csv ('Directiondata.csv')
print (directiondata)
# velocity is the mean velocity with which wind from this direction blows
# c determines how wide the distribution is
# frequency/100 determines how often the wind is blowing at this angle

In [ ]:
# could make comments here a markdown cell??

# interpolate 'velocity'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = directiondata['Angle']
y = directiondata['Velocity']
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
wbvel = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360,0.10)
plt.plot(x,y, label='Line')
plt.plot(xs, wbvel(xs), label="Interpolated line ")
plt.title('Interpolated Velocity Data')
plt.xlabel('Angle')
plt.ylabel('Velocity')
plt.show()

# blue lines shows linear interpolation, orange shows smooth interpolation including across the
# periodic boundary.

In [ ]:
#interpolate 'c'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = directiondata['Angle']
y = directiondata['c']
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
wbshape = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360, 0.1)
plt.plot(x,y)
plt.plot(xs, wbshape(xs), label="S")
plt.title('Interpolated Shape Parameter Data')
plt.xlabel('Angle')
plt.ylabel('Shape Parameter')
plt.show()

In [ ]:
#interpolate 'frequency'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = directiondata['Angle']
y = directiondata['frequency']/(100.*30.) #Data is in percent per 30 degrees, changing to probability per degree
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
windfreq = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360, 0.1)
plt.plot(x,y)
plt.plot(xs, windfreq(xs), label="S")
plt.title('Interpolated Probability Data')
plt.xlabel('Angle')
plt.ylabel('Probability')
plt.show()
# plot shows wind direction is predominantly 200-320deg.

In [ ]:
# Simple Attenuation fn
# Determines how much wind there is AFTER a turbine.
def att(dist, ang, model): 
        # 'Model' parameters are defined in 2ND CELL.

    # angular part
    angular=np.where(2.*model[1]*np.abs(ang)<np.pi,np.square(np.cos(model[1]*ang)),0.)
    # angular = np.cos(model[1]*ang WHEN 2.*model[1]*np.abs(ang) is less than pi, else angular = 0.
    
    # radial part (distance) (Gaussian Function)
    radial=np.exp(-np.square(dist/model[2])) # decreasing exponential of square, scaled by 2nd parameter
    penalty=np.exp(-np.square(dist/200))
    #penalty = 0
    return 1.0-1*model[0]*angular*radial-2*model[0]*penalty # OUTCOME

In [ ]:
#ADD a +-150 m sheer penalty either side along 0 degrees



azimuths = np.radians(np.linspace(0, 360, 360)) # angles theta from 0 to 360 degress in 1 degree increments. Grid
zeniths = np.arange(0, 4000, 50) # distance from 0 to 4000 broken up into increments of 50.

r, theta = np.meshgrid(zeniths, azimuths)
values = att(r,np.mod(theta+np.pi,np.pi*2)-np.pi,model) # gets angles from -pi to pi, not from 0 to 2pi.

#print(values)
#-- Plot... ------------------------------------------------
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
ax.contourf(theta, r, np.maximum(values,0.85*np.ones(np.shape(values))))

plt.show()
print(np.min(values))

## Markdown cell description:
# Polar plot is for 1 turbine, blue area shows where wind speed is affected. (Numbers label distance from turbine).
# Across the curve the distribution is the hump of cosine^2 function.
# Radially the distribution shown is Gaussian.
# wake is too narrow close to the turbine as it treats the turbine as a point rather than accommodating
# for the blades' diameter.

In [ ]:
# # isn't useful for continuing code, just rotates the windfarm.
# def rotate(angle,coords):
#     angle = np.pi*angle/180.
#     rotcoordx = []
#     rotcoordy = []
#     for coord in coords:
#         rotcoordx+=[coord[0]*np.cos(angle)-coord[1]*np.sin(angle)]
#         rotcoordy+=[coord[0]*np.sin(angle)+coord[1]*np.cos(angle)]
#     rotcoords=[rotcoordx,rotcoordy]
#     rotcoords=np.array(rotcoords).T   
#     return rotcoords 
# #print rotate(254,coords)[:,0]
# plt.scatter(rotate(0,coords)[:,0],rotate(0,coords)[:,1])

In [ ]:
# OPTIMISATION FUNCTION returning total deviation between model and training set.
#Total att
def deviation(b):
    total_att=np.ones((nturb,nwind))
    deviation=0
    for k in range(0,nwind):
        for j in range(0,nturb):
            for i in range(0,nturb):
                if (i!=j):
                    total_att[j,k] = total_att[j,k]*att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-wind[k])+np.pi,np.pi*2)-np.pi,b)
 #           print('{0} {1} {2}'.format(j,wind[k],total_att[j,k]))
            deviation=deviation+np.square(vref.get_group(wind[k]).iat[j,0]-v0*total_att[j,k])
    return deviation

In [ ]:
# OPTIMISATION FUNCTION returning deviation for each turbine.
#Total att
def deviation_sq(model):
    total_att=np.ones((nturb,nwind))
    dev_vec=np.zeros((nturb*nwind))
    for k in range(0,nwind):
        for j in range(0,nturb):
            for i in range(0,nturb):
                if (i!=j):
                    total_att[j,k] = total_att[j,k]* att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-wind[k])+np.pi,np.pi*2)-np.pi,model)
 #           print('{0} {1} {2}'.format(j,wind[k],total_att[j,k]))
            dev_vec[nturb*k+j]=(vref.get_group(wind[k]).iat[j,0]-v0*total_att[j,k])
    return dev_vec

In [ ]:
# USEFUL OUTPUT
# Calculates the % of wind that arrives at a turbine for a certain angle.
# Starts with assumption that there's zero wake so all turbines get full wind.
#talked about
#Total att
def windspeedreduction(distance,angle,direction,model): # vectors
    ndir=np.size(direction)
    total_att=np.ones((nturb,ndir)) # as many rows as there are turbines, as many columns as there are wind directions.
 #   power_vec=np.zeros((nturb,ndir))
    for j in range(0,nturb):
        for i in range(0,nturb):
            if (i!=j):
                # compounding wind speed reductions
                total_att[j,:] = total_att[j,:]* att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-direction[:])+np.pi,np.pi*2)-np.pi,model)
  #      power_vec[j,:]=(v0*total_att[j,:])**3
    return total_att # matrix output that's needed. n turbine rows, n direction columns.
                     # Tells how much of 100% of wind arrives at a certain turbine from a certain direction.

In [ ]:
# Calculates the power produced by turbines when it sees a certain wind speed at a certain angle. 3D matrix
def power(wsr,v): # wind speed reduction, velocity
    nvel=np.size(v)
    nangle=np.size(wsr,1)
    power_vec=np.zeros((nturb,nangle,nvel))
    power_vec=POvec(np.outer(wsr,v)).reshape(nturb,nangle,nvel) # np.outer takes every wsr element and individually multiplies it with every element of v.
    return power_vec

In [ ]:
# gives probability of having wind speed v at a certain angle.
def windspeedprobability(angles,v,d_angle,d_vel):
    nvel=np.size(v) 
    nangle=np.size(angles)
    wsprob=np.zeros((nangle,nvel))
    for i in range(angles.shape[0]):
        # Get Weibull parameters for angle and evaluate probability
        wsprob[i,:]=(wei(v[:],wbvel(angles[i]),wbshape(angles[i])))*windfreq(angles[i])*d_angle*d_vel
    return wsprob

In [ ]:
# Defines Weibull distribution (which shows the probability of any given wind speed occuring).

def wei(x,n,k): # convention to have in this order, scale parameter comes first (x) 
    u=n/gamma(1+1/k) #scaled wind speed
    return (k / u) * (x / u)**(k - 1) * np.exp(-(x / u)**k)
# k = Weibull shape parameter
# n = scale parameter
# x= value we are valuating

In [ ]:
def wind_dist(v,vm):
    return wei(v,vm,k)

In [ ]:
wind_dist(1.0,v0)

In [ ]:
# Plots Weibull distribution.

x=np.arange(0,25,0.1)
y=wind_dist(x,v0)

plt.xlabel('Wind Velocity')
plt.ylabel('Probability')
plt.plot(x,y)

In [ ]:
velocities=np.arange(Vc,Vf,dvel) # 1D array ranging from Vc to Vf in dvel intervals.
angles=np.arange(0,360,dang) # 0 to 360 with intervals of dang.
wsp=windspeedprobability(angles,velocities,dang,dvel) # tells probability of getting wind from a certain direction with a certain velocity.
# sum of all numbers would add up to 1.
print(wsp,np.shape(wsp),np.sum(wsp))
print(np.shape(velocities))
#plt.plot(x,y)

In [ ]:
# Calculates the power curve of a turbine.

# Plots the power curve of a turbine.
# No power output from 0-4m/s, then cubic rise up to 15m/s and stays
# constant until cut-off velocity of 25m/s.

#cubic Based Power output
a=Pr/(Vr**3-Vc**3)
b=Vc**3/(Vr**3-Vc**3)

def q(v): # q is power output

    if (v<Vc): # below cut in velocity power is 0.
        q=0
    elif (v<Vr):
        q=a*v**3-b*Pr # cubic power output between cut in and max.
    elif (v<Vf):
        q=Pr # max power between max and cut off velocity.
    else: 
        q=0 # no power above cut off velocity.
    return q

In [ ]:
x=np.arange(0,25,0.1)
POvec=np.vectorize(q) # vectorises power output function q (from cell above).
y=POvec(x)

plt.xlabel('Wind Velocity')
plt.ylabel('Power')
plt.plot(x,y)

In [ ]:
wsr=windspeedreduction(distance,angle,angles,model)
powout=power(wsr,velocities)
#print(powout,np.shape(powout))

In [ ]:
# Expected turbine output (Watts) in Horns Rev 1 for the given wind distributions
# and positions.Can be seen that top left (NW) produces most energy.

# EXPECTED POWER FROM WIND FARM CALCULATIONS
# multiply the power that you'd get for each wind angle and speed condition by the probability of this occuring, for all angles and speed
# values and sum to find the expected power.
# Sum up this value for all turbines to find total wind farm expected power output.
output=np.zeros((nturb)) 
output=np.tensordot(powout,wsp,2)
print(output,np.shape(output),np.sum(output))

# last output is total power output in Watts.

In [ ]:
# What all turbines would output (Watts) with no interference.
#Reference output:
# Same calculations as cell above except each turbine sees the full amount of wind every time
# (i.e. no wind reduction from other turbines).
refpowout=power(np.ones((nturb,np.size(angles))),velocities)
refoutput=np.zeros((nturb))
refoutput=np.tensordot(refpowout,wsp,axes=2)
print(refoutput,np.shape(refoutput),np.sum(refoutput))

# last output is total power output in Watts.

In [ ]:
# # assigns smaller interpolation intervals to dvel and dang to compare to coarser interpolations.
# dvel=0.1
# dang=0.1
# velocities=np.arange(Vc,Vf+dvel,dvel)
# angles=np.arange(0,360,dang)
# wsp=windspeedprobability(angles,velocities,dang,dvel)
# wsr=windspeedreduction(distance,angle,angles,model)
# powout=power(wsr,velocities)
# output=np.zeros((nturb)) 
# output=np.tensordot(powout,wsp,2)
# print (dvel,dang,np.sum(output))
# fineout=output

In [ ]:
# # Calculates RMS error from changing interpolation value.
# velarray=np.array([1.5])
# angarray=np.array([5])
# rmserror=np.zeros((np.size(velarray),np.size(angarray)))
# i=0
# for dvel in velarray:
#     j=0
#     for dang in angarray:
#         velocities=np.arange(Vc,Vf+dvel,dvel)
#         angles=np.arange(0,360,dang)
#         wsp=windspeedprobability(angles,velocities,dang,dvel)
#         wsr=windspeedreduction(distance,angle,angles,model)
#         powout=power(wsr,velocities)
#         output=np.zeros((nturb)) 
#         output=np.tensordot(powout,wsp,2)
#         rmserror[i,j]=np.sqrt(np.sum(np.square(output-fineout))/np.size(output))
#         print (dvel,dang,np.sum(output),rmserror[i,j])
#         j+=1
#     i+=1
# print(rmserror)

In [ ]:
# analyses the coordinates to collate the distances and angles between all of them
# calculates the minimum spanning tree length and adds the assumption of cabling costs,
# turbines costs, maintenance costs and fixed costs.

listofindiceslist = []
listofTotalCost = []
listofCostperWatt = []
listofpositionlist = []

def targetfunction(coord_flat):
#from scipy.optimize import minimize
    
    #These are used instead of res.x as res.x only holds the coordinates, not the other associated parameters, inc. the
    #optimal clustering, which will not necessarily be the same if the clustering algorithm is run again as the start
    #position is random for clustering. Additionally, res.x only provides an approximate minimization, whereas storing
    #values as global variables and finding the minimum guarantees that the minimum value found is returned.
    
    global listofindiceslist
    global listofTotalCost
    global listofCostperWatt
    global listofpositionlist

    
    
    arr1 = np.zeros((1,2),dtype=float)
    arr2=np.reshape(coord_flat,(-1,2)) # undoes the coordinate flattening
    positionlist = np.concatenate((arr1,arr2),axis=0)
    print(positionlist)

#     positionlist=np.reshape(coord_flat,(-1,2))
#     if abs(positionlist[0,0]) > 0 and abs(positionlist[0,1]) > 0:
#         arr1 = np.zeros((1,2),dtype=float)
#         positionlist = np.concatenate((arr1,positionlist),axis=0)
#         print('INSIDE THE IF STATEMENT')
    
    #np.appendleft(positionlist,[0,0])
    
    #positionlist = np.insert(positionlist, [0,:], [0,0])
    #print(np.shape(positionlist))
    
    nturb=np.size(positionlist,0)-1
    print(nturb)
    #clustering_algorithm(positionlist,nturb,maxnodespertree)

    indiceslist,u_labels,label = clustering_algorithm(positionlist,nturb,maxnodespertree)
    
    
    
    #Summeddistancetosub, min_index = sub_connection(positionlist)
#    print(coords)
    distance,angle,distance_global,angle_global = geom_analysis(positionlist,indiceslist)
    
    #print(np.shape(distance))
    
    MSTweight=Minimum_Spanning_Tree(distance,indiceslist)
    MSTWeightSum=0
    
    for a in range(0,math.ceil(nturb/maxnodespertree)):
        MSTWeightSum += MSTweight[a][0]
    
    #TotalCablingDistance = MSTWeightSum
    
    CableCost=(MSTWeightSum/1000)*300###
    
    DepthCostAll = depthvalues(positionlist)[0]
    
    # technically could add these at end as they add slight computation itme but this is neglibile. It's best to keep these
    # in as number of turbines can change. Especially if you then want to optimise the number of turbines.
    
    TurbineCost=(nturb)*3000000###
    MaintenanceCosts=(nturb)*2*1563720###
    FixedCost=600000###
    
    ########Set a total costs cap as part of objective 2
    
    
    #LandCost=(xmax-xmin)*(ymax-ymin)*(17245/10000)###
    
    LandCost = (max(positionlist[:,0])-min(positionlist[:,0])) * (max(positionlist[:,1])-min(positionlist[:,1])) * (17245/10000)
    
    
    #NEED TO ADD LAND COST, CABLE COST AND DEPTH COST BACK IN!
    
    #LandCost + DepthCostAllTurbines + CableCost
    
    TotalCost=TurbineCost+DepthCostAll+MaintenanceCosts+FixedCost+LandCost+CableCost### #ADD DEPTHCOST
    
    # remove penalty also? Redundant
    #penalty=0
    
    wsr=windspeedreduction(distance_global,angle_global,angles,model)
    powout=power(wsr,velocities) # total output power
    print(np.shape(wsp),np.shape(velocities),np.shape(powout))
    output=np.tensordot(powout,wsp,axes=2)
    
    print('Total power output in Watts', np.sum(output),'Total Cost in £', TotalCost, '£/W', (TotalCost/(np.sum(output))))###
# outputs cost per power e.g. £/watt.

# remove penalty also? Redundant
    print('Total C',TotalCost)
    print('Cable C', CableCost)
    print('Turbine C', TurbineCost)
    print('Land C', LandCost)
    print('Maint C', MaintenanceCosts)
    print('Fixed C', FixedCost)
    
    CostperWatt = TotalCost/(np.sum(output))
    
    listofindiceslist.append(indiceslist)
    listofTotalCost.append(TotalCost)
    listofCostperWatt.append(CostperWatt)
    listofpositionlist.append(positionlist)
    
    #if(np.min(distance)<200): penalty=10000*nturb*(np.min(distance)-200)**2
    return CostperWatt #+penalty


In [ ]:
#DO NOT RUN UNLESS NECESSARY!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


while True:  
    
    flat_coords=positionlist[1:,:].flatten() # turns 2D coords into single list of data without any more structure.


#CURRENTLY SET BOUNDS TO NONE AS IT WAS CAUSING ISSUES??
#bounds=Bounds(np.tile([-3000, -3000],nturb), np.tile([3000, 3000],nturb)) # sets MAX COORD VALUES
#print(bounds)

    dvel=1.5
    dang=5
    velocities=np.arange(Vc,Vf,dvel)
    angles=np.arange(0,360,dang)


## COULD USE BASINHOPPING AND SET THE STEP SIZE

    res = minimize(targetfunction, flat_coords, method='L-BFGS-B', jac=False, hess=None, options={'disp': 2, 'eps': 50})
    
    min_CostperWatt = min(listofCostperWatt)
    minimum_index = listofCostperWatt.index(min_CostperWatt)
    optimised_TotalCost = listofTotalCost[minimum_index]
    
    
    if optimised_TotalCost<=budget:
        if not res.success:
            print(res.status, res.message)
        break
    else:
        nturb = nturb - 1
        listofindiceslist = []
        listofTotalCost = []
        listofCostperWatt = []
        listofpositionlist = []
        positionlist = np.delete(positionlist, len(positionlist)-1, 0)

        ###############################
        # CANNOT USE CALLBACK as this only returns targetfunction output and not the other variables that are needed too.
        # NO EASY WAY TO RETURN VARIABLES NEEDED.
############################################################
#### MAKE THE MOST EFFICIENT WIND FARM UNDER BUDGET

        # NEED TO CHANGE TARGET FUNCTION TO GET MOST POWER OUT, USING THE BUDGET AS A CONSTRAINT FOR COST,
        # look into constraints for minimization function. CAN STILL USE SAME NTURB-1 LOGIC!
############################################################
        
        # NEED TO CHANGE TARGET FUNCTION TO GET MOST POWER OUT, THEN FIND THE MOST POWER OUT UNDERNEATH THE BUDGET COST.
        


In [ ]:
print('£/W:', min_CostperWatt)
print('Budget: £', budget)
print('Total Cost: £', optimised_TotalCost)
print('Total power output:',(optimised_TotalCost/min_CostperWatt),'W')
# USE THIS INDICESLIST OUTPUT TO SEE THE CLUSTERING TO MAKE THE MST

In [ ]:
print(listofindiceslist[minimum_index])

In [ ]:
new_indices_list = listofindiceslist[minimum_index]
newcoords = listofpositionlist[minimum_index]
print(newcoords)

In [ ]:
distance,angle,distance_global,angle_global = geom_analysis(newcoords,new_indices_list)
MSTweight = Minimum_Spanning_Tree(distance,new_indices_list)

links = [None] * math.ceil(nturb/maxnodespertree)
MSTWeightSum=0
    
mindepth = (depthvalues(newcoords))[1]
maxdepth = (depthvalues(newcoords))[2]


fig, ax = plt.subplots()
plt.axis('equal')
ax.set_xlabel('x distance [m]')
ax.set_ylabel('y distance [m]')
ax.set_xlim([min(newcoords[:,0])-100, max(newcoords[:,0])+100])
ax.set_ylim([min(newcoords[:,1])-100, max(newcoords[:,1])+100])
   
    
for a in range(0,len(MSTweight)):
    MSTWeightSum += MSTweight[a][0]
    links[a] = MSTweight[a][1]
    for b in links[a]:
         
        u = newcoords[int(new_indices_list[a,b[0]])][0], newcoords[int(new_indices_list[a,b[1]])][0]
        
        v = newcoords[int(new_indices_list[a,b[0]])][1], newcoords[int(new_indices_list[a,b[1]])][1]
        
        plt.plot(u,v,c='black')
    
CableCost=(MSTWeightSum/1000)*300###
    
DepthCostAll = depthvalues(newcoords)[0]

TurbineCost=(nturb)*3000000###
MaintenanceCosts=(nturb)*2*1563720###
FixedCost=600000###
    
LandCost = (max(newcoords[:,0])-min(newcoords[:,0])) * (max(newcoords[:,1])-min(newcoords[:,1])) * (17245/10000)
    
TotalCost=TurbineCost+DepthCostAll+MaintenanceCosts+FixedCost+LandCost+CableCost### #ADD DEPTHCOST

print('Total Cost: £', TotalCost, )
print('Total Power Output:', optimised_TotalCost/min_CostperWatt, 'W')
print('Cost per Watt',min_CostperWatt,'£/W')
print('Cabling Cost: £', CableCost)
print('Depth Cost: £', DepthCostAll)
print('Turbine Cost: £', TurbineCost)
print('Land Cost: £', LandCost)
print('Maintenance Cost: £', MaintenanceCosts)
print('Fixed Cost: £', FixedCost)


levels = np.linspace(mindepth-3,maxdepth+3,24) # Change to min(smooth) and max(smooth) somehow


tcf = ax.tricontourf(data['centered_easting'], data['centered_northing'], smooth(data['centered_easting'], data['centered_northing'], grid=False), levels=levels)
#plt.imshow(tcf, vmin=16, vmax=40)
cbar = plt.colorbar(tcf)
cbar.set_label('Depth (m)')
ax.set_title('Plot showing sea floor topography and turbines with cables');
ax.scatter(newcoords[:,0],newcoords[:,1],c='red')
ax.scatter(0,0,c='blue')